In [52]:
# Dependencies
import pymongo
import pandas as pd
from bs4 import BeautifulSoup as bs
import datetime as dt
import requests as req
import datetime as dt
from splinter import Browser
import time

#### Initialize pymongo client with connection through localhost to M2Mars_db, collection db.news for mars_news

In [54]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [42]:
# Define database and collection
db = client.M2Mars_db
collection = db.news

#### Access local nasa news content, open and read html, create news_soup with bs4 ####

In [43]:
# set local path for static html file provided by instr.
local_nasa_file = 'news_nasa_mars_exploration_program.html'

# open and read local html 
nasa_html = open(local_nasa_file, "r").read()

# create bs4 parsed html object
news_soup = bs(nasa_html, "html.parser")


#### Iterate through news_soup to access each attr, save to dictionary and insert to collection ####

In [44]:
# iterate through news_soup to scrape text date, title and teaser attrs in list_text class, return and
# create and insert collection ittems

for news in news_soup.find_all('div', attrs={'class': 'list_text'}):
    dte = news.find('div', attrs={'class': 'list_date'})
    title = news.find('div', attrs={'class': 'content_title'})
    teaser = news.find('div', attrs={'class': 'article_teaser_body'})
    # create post dict for insert to mongoDB document
    post = {
        'date': dte.text,
        'title': title.text,
        'teaser': teaser.text
    }
    #print(post)
    collection.insert_one(post)

#### Repeat db, collection, open, soup, iterate and insert processes for JPL featured photo ####

In [45]:
#### Go back to assignment to tackle splinter use in JPL photo workflow - N.B. initial bs4 workflow unsuccessful ####

# Define database and collection
db = client.M2Mars_db
collection = db.pic

jplPic = []

# create chrome browser open with local chromedriver .exe (or set path) and visit target jpl url
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless = False)
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)

# allow 2 sec for page load and click to nav to feature full image via splinter
time.sleep(2)
browser.click_link_by_partial_text('FULL IMAGE')

# allow 5 sec for page load and click to nav to more info
time.sleep(5)
browser.click_link_by_partial_text('more info')

# create next level down soup and access photo url components
picHtml = browser.html
picSoup = bs(picHtml, 'html.parser')
picUrl = picSoup.find('img', class_ = 'main_image')
# print(picUrl)
picSrc = picUrl.get('src')

# create feature photo url with src url from above
featured_image_url = "https://www.jpl.nasa.gov"+picSrc

jplPic = {'featured_image_url': featured_image_url}
collection.insert_one(jplPic)

browser.quit()
# print(featured_image_url)

#### Repeat db, collection, open, soup, iterate and insert processes for NASA Mars twitter for weather report ####

In [46]:
# reiterate database and redefine collection for NASA Mars twitter weather report
db = client.M2Mars_db
collection = db.marswx

In [47]:
# Identify and request content for mares weather twitter acct
marswx = 'https://twitter.com/marswxreport?lang=en'
response = req.get(marswx)

# Create BeautifulSoup object, parse with 'html.parser'
wxsoup = bs(response.text, 'html.parser')

#print(wxsoup.prettify())

#### Iterate through wxsoup to access each attr, save to dictionary and insert to collection ####

In [48]:
# locate li tag, create sub of soup and iterate through tweets to scrape handle, tweet text -- return,
# create and insert collection records to mongoDB, dictionary and recent wxrpt

wxdict = []
wxtweets = wxsoup.find_all('li','js-stream-item')
for wx in wxtweets:
        wx.find('p','tweet-text')
        wx_user = wx.find('span','username').text
        wx_text = wx.find('p','tweet-text').text
        wxtimestamp = wx.find('a','tweet-timestamp')['title']
        wxdict.append({'handle': wx_user, 'wxrpt': wx_text, 'wxtime': wxtimestamp})
        post = {
            'handle': wx_user,
            'wxrpt': wx_text,
            'wxtime': wxtimestamp
        }
        #collection.insert_one(post)
        # verify tweet has a wx and break if true, insert post
        if "Sol " in wx_text:
            mars_weather = wx_text
            collection.insert_one(post)
            break
            
#mars_weather       

#### Collect space-facts.com mars tabular data to pandasDF and output html table string ####

In [49]:
# read space-facts/mars html into DF
url = ('https://space-facts.com/mars/')
match = 'Equatorial Diameter:'
facts = pd.read_html(url, match = match)
factsDF = facts[0]

# factsDF

In [50]:
# print and output factsDF data to html table
with open('mars_facts.html', 'w') as mf:
    factsDF.to_html(mf)
    
# print(factsDF.to_html()) 

#### Collect Mars Hemishere image urls and titles from USGS page, create dictionary of img_url and title ####

In [51]:
# reiterate database and redefine collection USGS hemisphere photo urls
db = client.M2Mars_db
collection = db.hemis

# identify USGS mars hemishpere resource
usgs_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

# Retrieve page with requests
response = req.get(usgs_url)

# Create BeautifulSoup object, parse with 'html.parser'
usgsSoup = bs(response.text, 'html.parser')

# Set list for data collection and dictionary creation
hemisphere_image_urls = []

# Soup for 4 hemispheres, i.e. landing page target class
fourHSoup = usgsSoup.find_all('a', class_='itemLink product-item') 

#iterate through soup to capture image title, build url to hemishphere page, retrieve image src and load dictionary
for item in fourHSoup:
    urlBack = item['href']
    title = item.find('div', attrs={'class': 'description'}).text
    urlTo = "https://astrogeology.usgs.gov"+urlBack
    #print(title, urlTo)
    # follow urlTo down a level with get -- create soup for 1 hemisphere per iter,  write dict and insert to collection
    response = req.get(urlTo)
    oneHSoup = bs(response.text, 'html.parser')
    findSrc = oneHSoup.find('div', class_ ='downloads')
    oneUrl = findSrc.find('a')['href']
    post = {
        'title': title,
        'img_url': oneUrl
    }
    hemisphere_image_urls.append({'title': title, 'img_url': oneUrl})
    collection.insert_one(post)
#hemisphere_image_urls  
